<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [18]:
from utils2 import MultiSig
from talib import BBANDS

gazp = pd.read_csv("gazp.csv", parse_dates=["Date"]).sort_values("Date")
gazp = gazp[:150]
date = gazp['Date']
price = gazp["Close"].values
gazp['upper'], gazp['middle'], gazp['lower'] = BBANDS(price, 20, 2, 2, matype=0)

In [48]:
enter_long = (('Close','lower','crossup',False),)
exit_long = (
    ('Close','middle','crossup',False),
    'or_',
    ('Close','lower','crossdown',False)
)

In [49]:
enter_short = (('Close','upper','crossdown',False),)
exit_short = (
    ('Close','middle','crossdown',False),
    'or_',
    ('Close','upper','crossup',False)
)

In [21]:
def parse_sig(sig):
    L = len(sig)
    out = ()
    for i in range(0, L, 2):
            out += ((sig[i], sig[i + 1]),) if i < L - 1 else ((sig[i], "NOP"),)
    return out

parse_sig((enter_long,))

((('Close', 'lower', 'crossup', False), 'NOP'),)

In [69]:
def check_sig(sig, df, debug):
    '''
    1. Tuple of tuples
    2. Odd elements are tuples of 4, first 2 are cols, 3rd is 'how', 4th is bool
    3. Evens are either 'and_' or 'or_'
    '''
    if (not isinstance(sig, tuple)) or (not isinstance(sig[0], tuple)):
        raise ValueError(f"Signal {sig} must be tuple of tuples")
    for i, el in enumerate(sig):
        if i%2 == 0:
            if len(el) != 4:
                raise ValueError(f"Signal {el} doesn't have 4 elements")
            cols = df.columns
            if el[0] not in cols:
                raise ValueError(f"'{el[0]}' in {sig} is not a column supplied df")
            if el[1] not in cols:
                raise ValueError(f"'{el[1]}' in {sig} is not a column supplied df")
            if el[2] not in ['crossup','crossdown','lt_','gt_']:
                raise ValueError(f"'{el[2]}' in {sig} must be one of 'crossup','crossdown', 'lt', or 'gt'")
            if not isinstance(el[3], bool):
                raise ValueError(f"'{el[3]}' in {sig} must be boolean")
        else:
            if el not in ['and_','or_']:
                raise ValueError(f"Logic '{el}' in {sig} must be one of 'and_' or 'or_'")
    if debug:
        print(f"Signal {sig} is correctly defined")
    
def parse_sig(sig, df, debug):
    check_sig(sig, df, debug)
    L = len(sig)
    out = ()
    for i in range(0, L, 2):
        S = (df[sig[i][0]].values,df[sig[i][1]].values,sig[i][2],sig[i][3])
        out += ((S, sig[i + 1]),) if i < L - 1 else ((S, "NOP"),)
    return out

In [ ]:
class Strategy:
    
    def __init__(self, df, enter_long, exit_long, enter_short, exit_short, commission=.0003, debug=False):
        self.df = df
        self.enter_long = parse_sig(enter_long, df, debug)
        self.exit_long = parse_sig(exit_long, df, debug)
        self.enter_short = parse_sig(enter_short, df, debug)
        self.exit_short = parse_sig(exit_short, df, debug)
        self.commission = commission
    
    @staticmethod
    def check_sig(sig, df, debug):
    '''
    1. Tuple of tuples
    2. Odd elements are tuples of 4, first 2 are cols, 3rd is 'how', 4th is bool
    3. Evens are either 'and_' or 'or_'
    '''
    if (not isinstance(sig, tuple)) or (not isinstance(sig[0], tuple)):
        raise ValueError(f"Signal {sig} must be tuple of tuples")
    for i, el in enumerate(sig):
        if i%2 == 0:
            if len(el) != 4:
                raise ValueError(f"Signal {el} doesn't have 4 elements")
            cols = df.columns
            if el[0] not in cols:
                raise ValueError(f"'{el[0]}' in {sig} is not a column supplied df")
            if el[1] not in cols:
                raise ValueError(f"'{el[1]}' in {sig} is not a column supplied df")
            if el[2] not in ['crossup','crossdown','lt_','gt_']:
                raise ValueError(f"'{el[2]}' in {sig} must be one of 'crossup','crossdown', 'lt', or 'gt'")
            if not isinstance(el[3], bool):
                raise ValueError(f"'{el[3]}' in {sig} must be boolean")
        else:
            if el not in ['and_','or_']:
                raise ValueError(f"Logic '{el}' in {sig} must be one of 'and_' or 'or_'")
    if debug:
        print(f"Signal {sig} is correctly defined")
        
    
    @staticmethod
    def parse_sig(sig, df, debug):
        check_sig(sig, df, debug)
        L = len(sig)
        out = ()
        for i in range(0, L, 2):
            S = (df[sig[i][0]].values,df[sig[i][1]].values,sig[i][2],sig[i][3])
            out += ((S, sig[i + 1]),) if i < L - 1 else ((S, "NOP"),)
        return out
    
    @njit
    def run(self):
        enter_long = Signal(self.enter_long)
        exit_long = Signal(self.exit_long)
        enter_short = Signal(self.enter_short)
        exit_short = Signal(self.exit_short)

        L = self.price.shape[0]
        out = np.empty(L)
        out[:2] = 0.0
        out[-2:] = 0.0
        i = 1

        while i < (L-2):
            

            out[i+1] = 0.0

            if enter_long.sig(i) and i < (L-2):
                out[i+1] = self.price[i+2]/(self.price[i+1]*(1+self.commission)) - 1.0
                trade = True
                while trade and i < (L-3):
                    i += 1
                    out[i+1] = self.price[i+2]/self.price[i+1] - 1.0
                    if exit_long.sig(i):
                        trade = False
                        out[i+1] = (1-self.commission)*self.price[i+2]/self.price[i+1] - 1.0


            if enter_short(i) and i < (L-2):
                out[i+1] = (1-self.commission)*self.price[i+1]/self.price[i+2] - 1.0
                trade = True
                while trade and i < (L-3):
                    i += 1
                    out[i+1] = self.price[i+1]/self.price[i+2] - 1.0
                    if exit_short(i):
                        trade = False
                        out[i+1] = self.price[i+1]/(self.price[i+2]*(1+self.commission)) - 1.0


            i +=1
        return out